In [24]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

img_height = 300
img_width = 300

K.clear_session() 

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=2,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], 
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.5,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

#학습된 weight의 경로를 지정
weights_path = 'C:\\Users\\user\\keras\\ssd_keras\\ssd300_pascal_07+12_epoch-08_loss-1.9471_val_loss-1.9156.h5'

model.load_weights(weights_path, by_name=True)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [25]:
#영상의 경로를 지정하고 프레임 캡쳐
#나중에는 비디오 캡쳐를 함과 동시에 input_images리스트에 곧바로 넣어버려서, 불필요한 이미지 입출력 과정을 줄이자
#1초에 4프레임 캡쳐로 바꿈 (6프레임마다 저장)
vidcap = cv2.VideoCapture('C:\\Users\\user\Desktop\\2018_09_22_10_30_19.mp4')
success,imagefile = vidcap.read()
count = 0
while success:
    if(count%6==0):
#프레임 캡쳐를 저장할 경로
        cv2.imwrite("C:\\Users\\user\Desktop\\frames\\frame%d.jpg" % count, imagefile)    
    success,imagefile = vidcap.read()
    count += 1

orig_images = [] 
input_images = [] 

# range는 추후 변수를 사용할 수 있도록 변경
for i in range(0,320):
    if(i%6==0):
#프레임 캡쳐를 불러오는 경로
        img_path = 'C:\\Users\\user\Desktop\\frames\\frame%d.jpg'%i
        #print(img_path)
        orig_images.append(imread(img_path))
        img = image.load_img(img_path, target_size=(img_height, img_width))
        img = image.img_to_array(img)
        img = np.array(img)
        input_images.append(img)
        
input_images = np.array(input_images)
orig_images = np.array(orig_images)

num_of_frames = 16
counting = 0
saving_bounding_boxes = []

print("Predicted boxes:\n")
print('   class   conf xmin   ymin   xmax   ymax')

# range는 추후 변수를 사용할 수 있도록 변경
for i in range(0, 4):
    y_pred = model.predict(input_images[i*num_of_frames:i*num_of_frames + num_of_frames])
    confidence_threshold = 0.4

    y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
    np.set_printoptions(precision=2, suppress=True, linewidth=90)

    for j in range(0, num_of_frames):
        print('frame :',counting)
#        print(y_pred_thresh[j])
        for box in y_pred_thresh[j]:
            # Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
            xmin = box[2] * orig_images[0].shape[1] / img_width
            ymin = box[3] * orig_images[0].shape[0] / img_height
            xmax = box[4] * orig_images[0].shape[1] / img_width
            ymax = box[5] * orig_images[0].shape[0] / img_height
            print('xmin : ',xmin, '  ymin : ',ymin, '  xmax : ',xmax, '  ymax : ',ymax)
            # 균열이 탐지된 프레임과 b-box 정보가 saving_bounding_boxes <- 여기에 저장됨
            saving_bounding_boxes.append([counting, xmin,ymin,xmax,ymax])
        counting += 6
            
'''
5. 방향 설정까지 했다면, 그 방향에서 edge가 몇 픽셀 떨어져 있는지를 재고, 각 skeleton line상의 pixel에 그 정보를 저장하자!
6. 거기서 max값을 return하자
'''

Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
frame : 0
xmin :  30.546334126790363   ymin :  -5.269902976353963   xmax :  833.9878662109375   ymax :  146.71426060994466
frame : 5
xmin :  28.93269266764323   ymin :  -5.153070624669393   xmax :  834.6906510416667   ymax :  145.89585647583007
frame : 10
xmin :  30.45083017985026   ymin :  -4.708958299954732   xmax :  834.2275048828125   ymax :  145.32056986490886
frame : 15
xmin :  29.221277516682942   ymin :  -3.926287325223287   xmax :  831.7132828776041   ymax :  145.25722096761066
frame : 20
xmin :  26.708422241210936   ymin :  -3.906070903937022   xmax :  833.4325048828125   ymax :  147.31289418538412
frame : 25
xmin :  27.74120086669922   ymin :  -4.354046646753947   xmax :  835.6152180989583   ymax :  147.0047472635905
frame : 30
xmin :  20.11531529744466   ymin :  -5.203345465660095   xmax :  838.9105517578125   ymax :  147.78603032430013
frame : 35
xmin :  25.916158396402995   ymin :  0.14628484845161438   xmax :  8

'\n1. 어떻게 skeletion의 direction을 추출할 것인가 -> 가장 정확한 방법을 찾아야 함 -> 잘못 세면 overestimation 되기 십상\n2. direction만 잘 잡는다면 width를 계산하는 것은 오히려 어렵지 않다\n3. skeleton을 추출한 이미지와 edge를 추출한 이미지에 함께 접근해가며 픽셀마다 width를 매기면 됨\n4. 여기서 수 많은 width들 중 최종 width는 어떻게 정할 것인지가 또 다른 문제가 됨\n'

In [26]:
from skimage import io

cropped_frames = []

for i in range(0, len(saving_bounding_boxes)):
    frame_count = saving_bounding_boxes[i][0]//5
    frame = orig_images[frame_count]
    if(saving_bounding_boxes[i][1] < 0):
        saving_bounding_boxes[i][1] = 0
    xmin = int(saving_bounding_boxes[i][1])
    if(saving_bounding_boxes[i][2] < 0):
        saving_bounding_boxes[i][2] = 0
    ymin = int(saving_bounding_boxes[i][2])
    xmax = int(saving_bounding_boxes[i][3])
    ymax = int(saving_bounding_boxes[i][4])
    print(xmin,ymin,xmax,ymax)
    cropped_frame = orig_images[frame_count][ymin:ymax, xmin:xmax, :]
    cropped_frame = cropped_frame.astype('uint8')
    img_path = '../../Desktop/test/%d.jpg'%frame_count
    print(img_path)
    cropped_frames.append(cropped_frame)
    io.imsave(img_path, cropped_frame)

30 0 833 146
../../Desktop/test/0.jpg
28 0 834 145
../../Desktop/test/1.jpg
30 0 834 145
../../Desktop/test/2.jpg
29 0 831 145
../../Desktop/test/3.jpg
26 0 833 147
../../Desktop/test/4.jpg
27 0 835 147
../../Desktop/test/5.jpg
20 0 838 147
../../Desktop/test/6.jpg
25 0 835 163
../../Desktop/test/7.jpg
30 0 837 153
../../Desktop/test/8.jpg
31 0 839 152
../../Desktop/test/9.jpg
33 0 837 153
../../Desktop/test/10.jpg
31 0 836 153
../../Desktop/test/11.jpg
36 1 836 159
../../Desktop/test/12.jpg
31 0 838 151
../../Desktop/test/13.jpg
28 0 838 146
../../Desktop/test/14.jpg
31 0 838 145
../../Desktop/test/15.jpg
28 0 838 145
../../Desktop/test/16.jpg
25 0 838 145
../../Desktop/test/17.jpg
24 0 835 146
../../Desktop/test/18.jpg
21 0 837 145
../../Desktop/test/19.jpg
9 0 830 145
../../Desktop/test/20.jpg
12 0 833 145
../../Desktop/test/21.jpg
19 0 841 143
../../Desktop/test/22.jpg
17 0 840 143
../../Desktop/test/23.jpg
18 0 841 140
../../Desktop/test/24.jpg
13 0 840 143
../../Desktop/test/25.j

In [27]:
# 1. Image binarization(Sauvola's method) using Pw and Pl, respectively
# 오래 걸리는 문제가 있음

import time
import matplotlib
import matplotlib.pyplot as plt
import cv2
from skimage import io
from skimage import data
from skimage.color import rgb2gray
from skimage.data import page
from skimage.filters import (threshold_sauvola)
from PIL import Image

start_time = time.time() 
matplotlib.rcParams['font.size'] = 9

sauvola_frames_Pw_bw = []
sauvola_frames_Pl_bw = []
sauvola_frames_Pw = []
sauvola_frames_Pl = []

# Upload the image
for i in range(0,len(cropped_frames)):
    img = cropped_frames[i]
    img_gray = rgb2gray(img)

    # 논문에선 각각 70,180이었으나 여기선 홀수 input만 가능
    window_size_Pw = 71
    window_size_Pl = 181
    thresh_sauvola_Pw = threshold_sauvola(img_gray, window_size=window_size_Pw, k=0.42)
    thresh_sauvola_Pl = threshold_sauvola(img_gray, window_size=window_size_Pl, k=0.18)

    #Below are the converted images through Sauvola's method.
    # _bw will contain 0 or 1, not true or false. bw means black or white.
    binary_sauvola_Pw = img_gray > thresh_sauvola_Pw
    binary_sauvola_Pl = img_gray > thresh_sauvola_Pl
    binary_sauvola_Pw_bw = img_gray > thresh_sauvola_Pw
    binary_sauvola_Pl_bw = img_gray > thresh_sauvola_Pl

    binary_sauvola_Pw_bw.dtype = 'uint8'
    binary_sauvola_Pl_bw.dtype = 'uint8'

    binary_sauvola_Pw_bw *= 255
    binary_sauvola_Pl_bw *= 255
    
    sauvola_frames_Pw_bw.append(binary_sauvola_Pw_bw)
    sauvola_frames_Pl_bw.append(binary_sauvola_Pl_bw)
    sauvola_frames_Pw.append(binary_sauvola_Pw)
    sauvola_frames_Pl.append(binary_sauvola_Pl) 
    
    img_path_Pw = '../../Desktop/Sauvola/Sauvola_Pw_%d.jpg'%i
    img_path_Pl = '../../Desktop/Sauvola/Sauvola_Pl_%d.jpg'%i
    
    io.imsave(img_path_Pw, binary_sauvola_Pw_bw)
    io.imsave(img_path_Pl, binary_sauvola_Pl_bw)

print("start_time", start_time)
print("--- %s seconds ---" %(time.time() - start_time))

c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\util\arraycrop.py:177: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropped = ar[slices]


start_time 1537672974.251778
--- 271.1478292942047 seconds ---


In [28]:
# 2. Extract the skeletons of each images

from skimage.morphology import skeletonize
from skimage.util import invert

skeleton_frames_Pw = []
skeleton_frames_Pl = []

for i in range(0,len(cropped_frames)):
# Invert the binarized images
    img_Pw = invert(sauvola_frames_Pw[i])
    img_Pl = invert(sauvola_frames_Pl[i])

    # Below are skeletonized images
    skeleton_Pw = skeletonize(img_Pw)
    skeleton_Pl = skeletonize(img_Pl)

    # Convert true/false to 1/0 to save it as image
    skeleton_Pw.dtype = 'uint8'
    skeleton_Pl.dtype = 'uint8'

    skeleton_Pw *= 255
    skeleton_Pl *= 255

    skeleton_frames_Pw.append(skeleton_Pw)
    skeleton_frames_Pl.append(skeleton_Pl)   
    
    img_path_Pw = "../../Desktop/Skeleton/skeleton_Pw_%d.jpg"%i
    img_path_Pl = "../../Desktop/Skeleton/skeleton_Pl_%d.jpg"%i
    io.imsave(img_path_Pw, skeleton_Pw)
    io.imsave(img_path_Pl, skeleton_Pl)
    

c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pw_46.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pw_47.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pl_47.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pw_48.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\users\user\anaconda2\envs\tensorflow-gpu\lib\site-packages\skimage\io\_io.py:140: UserWarning: ../../Desktop/Skeleton/skeleton_Pw_49.jpg is a low contrast image


In [29]:
# 3. Detect the edges of each images
### edge detection 할 때, 좋은 parameter를 찾아야 한다. 지금은 edge가 너무 두꺼움 (overestimation됨) ###
import numpy as np
from scipy import ndimage as ndi
from skimage import feature

edges_frames_Pw = []
edges_frames_Pl = []

for i in range(0,len(cropped_frames)):
    # Compute the Canny filter for two values of sigma
    # canny(image, sigma=1.0, low_threshold=None, high_threshold=None, mask=None, use_quantiles=False)
    # sigma가 1이었으나, 0.1로 조정하여 실제 균열 edge와 거의 같게 만듦.
    # 정확도에서 문제가 생긴다면 1. skeleton의 방향 설정 방법을 바꾸던가, 2. 여기서 시그마 값을 살짝 늘리거나 줄여가면서 정확도를 테스트 해볼 것
    edges_Pw = feature.canny(sauvola_frames_Pw[i], 0.09)
    edges_Pl = feature.canny(sauvola_frames_Pl[i], 0.09)

    edges_Pw.dtype = 'uint8'
    edges_Pl.dtype = 'uint8'

    edges_Pw *= 255
    edges_Pl *= 255

    edges_frames_Pw.append(edges_Pw)
    edges_frames_Pl.append(edges_Pl)
    
    img_path_Pw = "../../Desktop/edges/edges_Pw_%d.jpg"%i
    img_path_Pl = "../../Desktop/edges/edges_Pl_%d.jpg"%i
    
    io.imsave(img_path_Pw, edges_Pw)
    io.imsave(img_path_Pl, edges_Pl)


In [148]:
np.set_printoptions(threshold=np.inf)
print(skeleton_frames_Pw[0].shape)

crop_skeleton = np.zeros((120,146))
crop_edges = np.zeros((120,146))

for i in range(0,120):
    for j in range(0,146):
        if(skeleton_frames_Pw[0][i][j] == 255): crop_skeleton[i][j] = crop_edges[i][j] = 1
        else: crop_skeleton[i][j] = crop_edges[i][j] = 0
        
        
np.savetxt("../../Desktop/skeleton.txt", crop_skeleton, '%d')
np.savetxt("../../Desktop/edge.txt", crop_edges, '%d')


'''
count = 0
for i in range(skeleton_frames_Pw[0].shape[0]):
    for j in range(skeleton_frames_Pw[0].shape[1]):
        if(skeleton_frames_Pw[0][i][j] == 255): count+=1
            
print(count)
'''

#edges_frames_Pw[0]

(146, 803)
1477


In [177]:
#Crack만이 detection되어서 넘어왔다는 가정이 있어야 함. 아니면 외부 배경 이미지도 균열 계산에 포함 됨

import queue
import math

#5픽셀이 기준 or above
dx_dir_right = [-5,-5,-5,-4,-3,-2,-1,0,1,2,3,4,5,5]
dy_dir_right = [0,1,2,3,4,5,5,5,5,5,4,3,2,1]

dx_dir_left = [5,5,5,4,3,2,1,0,-1,-2,-3,-4,-5,-5]
dy_dir_left = [0,-1,-2,-3,-4,-5,-5,-5,-5,-5,-4,-3,-2,-1]

dx_bfs = [-1,-1,0,1,1,1,0,-1]
dy_bfs = [0,1,1,1,0,-1,-1,-1]

start = [0,0]
next = []
q = queue.Queue()
q.put(start)

len_x = skeleton_frames_Pw[0].shape[0]
len_y = skeleton_frames_Pw[0].shape[1]

visit = np.zeros((len_x,len_y))
count = 0
pi = 3.14159

while(q.empty() == 0):
    next = q.get()
    x = next[0]
    y = next[1]
    right_x = right_y = left_x = left_y = -1
    
    
    if(skeleton_frames_Pw[0][x][y] == 255):
        for i in range(0, len(dx_dir_right)):
            right_x = x + dx_dir_right[i]
            right_y = y + dy_dir_right[i]
            if(right_x<0 or right_y<0 or right_x>=len_x or right_y>=len_y): 
                right_x = right_y = -1
                continue;
            if(skeleton_frames_Pw[0][right_x][right_y] == 255): break;

        if(right_x == -1): 
            right_x = x
            right_y = y

        for i in range(0, len(dx_dir_left)):
            left_x = x + dx_dir_left[i]
            left_y = y + dy_dir_left[i]
            if(left_x <0 or left_y<0 or left_x >=len_x or left_y>=len_y): 
                left_x = left_y = -1
                continue;
            if(skeleton_frames_Pw[0][left_x][left_y] == 255): break;

        if(left_x == -1): 
            left_x = x
            left_y = y

        base = right_y - left_y
        height = right_x - left_x
        hypotenuse = math.sqrt(base*base + height*height)
        
        if(base==0 and height != 0): theta = 90.0
        else: theta = math.degrees(math.acos((base * base + hypotenuse * hypotenuse - height * height)/(2.0 * base * hypotenuse)))
        
        theta += 90
        #탐색 a*cos + a*sin 공식으로 edge탐색 (양방향으로), 255가 걸리면 그 픽셀까지의 거리 재기
        a = 1
        while(1):
            pixel_y = a*math.cos(theta/pi)
            pixel_x = a*math.sin(theta/pi)
            print(pixel_y, pixel_x)
            if(edges_frames_Pw[0][int(pixel_y)][int(pixel_x)]==255):
                dist_1 = math.sqrt((y-int(pixel_y))**2 + (x-int(pixel_x))**2)
                print('end')
                break;
            a+=1

        a = 1
        theta += 180
        
        while(1):
            pixel_y = a*math.cos(theta/pi)
            pixel_x = a*math.sin(theta/pi)
            print(pixel_y, pixel_x)
            if(edges_frames_Pw[0][int(pixel_y)][int(pixel_x)]==255):
                dist_2 = math.sqrt((y-int(pixel_y))**2 + (x-int(pixel_x))**2)
                break;
            a+=1
            
        dist = dist_1 + dist_2
        print('dist : ', dist)
        #해당 위치와 균열 폭을 힘께 저장하는 새로운 리스트 사용하기
        
    
    for i in range(0,8):
        next_x = x + dx_bfs[i]
        next_y = y + dy_bfs[i]
        
        if(next_x<0 or next_y<0 or next_x>=len_x or next_y>=len_y): continue;
        if(visit[next_x][next_y] == 0): 
            q.put([next_x,next_y])
            visit[next_x][next_y] = 1
    

0.5814611725630805 -0.8135741544577036
1.162922345126161 -1.6271483089154073
1.7443835176892415 -2.440722463373111
end
0.979443442959183 -0.201718967983335
1.958886885918366 -0.40343793596667
2.938330328877549 -0.605156903950005
3.917773771836732 -0.80687587193334
4.897217214795915 -1.0085948399166749
5.876660657755098 -1.21031380790001
6.856104100714281 -1.412032775883345
7.835547543673464 -1.61375174386668
8.814990986632647 -1.8154707118500149
9.79443442959183 -2.0171896798333497
10.773877872551012 -2.218908647816685
11.753321315510195 -2.42062761580002
12.73276475846938 -2.622346583783355
13.712208201428561 -2.82406555176669
14.691651644387743 -3.025784519750025
15.671095087346927 -3.22750348773336
16.65053853030611 -3.429222455716695
17.629981973265295 -3.6309414237000297
dist :  220.88095529292582
0.5814611725630805 -0.8135741544577036
1.162922345126161 -1.6271483089154073
1.7443835176892415 -2.440722463373111
end
0.979443442959183 -0.201718967983335
1.958886885918366 -0.403437935

-0.12626243819319866 14.999468583809945
-0.13467993407274523 15.99943315606394
-0.1430974299522918 16.999397728317938
-0.15151492583183837 17.99936230057193
-0.15993242171138497 18.99932687282593
-0.16834991759093154 19.999291445079926
-0.1767674134704781 20.99925601733392
-0.18518490935002468 21.999220589587917
-0.19360240522957126 22.999185161841915
-0.20201990110911783 23.99914973409591
-0.21043739698866443 24.999114306349906
-0.218854892868211 25.999078878603903
-0.22727238874775757 26.9990434508579
-0.23568988462730414 27.999008023111895
-0.2441073805068507 28.998972595365892
-0.2525248763863973 29.99893716761989
-0.26094237226594386 30.998901739873883
-0.26935986814549046 31.99886631212788
-0.27777736402503705 32.998830884381874
-0.2861948599045836 33.998795456635875
-0.2946123557841302 34.99876002888987
-0.30302985166367674 35.99872460114386
-0.31144734754322334 36.998689173397864
-0.31986484342276994 37.99865374565186
-0.3282823393023165 38.99861831790585
-0.3366998351818631 39

-1.9191890605366195 227.99192247391116
-1.927606556416166 228.99188704616515
-1.9360240522957126 229.99185161841913
-1.9444415481752593 230.99181619067315
-1.9528590440548057 231.99178076292714
-1.9612765399343524 232.99174533518112
-1.969694035813899 233.99170990743514
-1.9781115316934454 234.99167447968912
-1.986529027572992 235.9916390519431
-1.9949465234525388 236.99160362419713
-2.003364019332085 237.9915681964511
-2.0117815152116316 238.9915327687051
-2.0201990110911785 239.9914973409591
-2.028616506970725 240.9914619132131
-2.0370340028502714 241.9914264854671
-2.0454514987298182 242.9913910577211
-2.0538689946093647 243.9913556299751
-2.062286490488911 244.99132020222908
-2.070703986368458 245.9912847744831
-2.0791214822480044 246.99124934673708
-2.087538978127551 247.99121391899106
-2.0959564740070977 248.99117849124508
-2.104373969886644 249.99114306349907
-2.1127914657661906 250.99110763575305
-2.1212089616457375 251.99107220800707
-2.129626457525284 252.99103678026106
-2.13

-3.73736817051868 443.98427008077437
-3.7457856663982265 444.98423465302835
-3.7542031622777734 445.98419922528234
-3.76262065815732 446.9841637975363
-3.7710381540368663 447.9841283697903
-3.779455649916413 448.9840929420443
-3.7878731457959596 449.98405751429834
-3.796290641675506 450.98402208655233
-3.804708137555053 451.9839866588063
-3.8131256334345993 452.9839512310603
-3.8215431293141457 453.9839158033143
-3.8299606251936926 454.9838803755683
-3.838378121073239 455.9838449478223
-3.8467956169527855 456.9838095200763
-3.855213112832332 457.9837740923303
-3.863630608711879 458.9837386645843
-3.8720481045914252 459.98370323683827
-3.8804656004709717 460.98366780909225
-3.8888830963505185 461.9836323813463
-3.897300592230065 462.9835969536003
-3.9057180881096114 463.98356152585427
-3.9141355839891583 464.98352609810826
-3.9225530798687047 465.98349067036224
-3.930970575748251 466.98345524261623
-3.939388071627798 467.9834198148703
-3.9478055675073445 468.98338438712426
-3.9562230633

-5.597634759898473 664.9764405489075
-5.60605225577802 665.9764051211615
-5.614469751657567 666.9763696934156
-5.622887247537113 667.9763342656695
-5.63130474341666 668.9762988379235
-5.639722239296207 669.9762634101775
-5.648139735175753 670.9762279824315
-5.6565572310553 671.9761925546854
-5.6649747269348465 672.9761571269395
-5.6733922228143925 673.9761216991935
-5.681809718693939 674.9760862714475
-5.690227214573486 675.9760508437015
-5.698644710453032 676.9760154159554
-5.707062206332579 677.9759799882095
-5.715479702212126 678.9759445604635
-5.723897198091672 679.9759091327174
-5.732314693971219 680.9758737049715
-5.740732189850766 681.9758382772254
-5.749149685730312 682.9758028494795
-5.757567181609859 683.9757674217334
-5.7659846774894055 684.9757319939874
-5.7744021733689515 685.9756965662415
-5.782819669248498 686.9756611384954
-5.791237165128045 687.9756257107495
-5.799654661007591 688.9755902830034
-5.808072156887138 689.9755548552574
-5.816489652766685 690.9755194275115
-

IndexError: index 803 is out of bounds for axis 0 with size 803